# Optimization

## Preprocessing

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv('/content/drive/My Drive/UC Berkeley Data Analytics Bootcamp/Module 21/Challenge/charity_data_preprocessed.csv')
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


### AFFILIATION

In [3]:
# Look at AFFILIATION value counts for binning
application_df['AFFILIATION'].value_counts()

Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

In [4]:
# Choose a cutoff value and create a list of classifications to be replaced

affiliations_to_replace = list(application_df['AFFILIATION'].value_counts().loc[lambda x: x < 15000].index)

# Replace in dataframe
for cls in affiliations_to_replace:
    application_df['AFFILIATION'] = application_df['AFFILIATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['AFFILIATION'].value_counts()

Independent         18480
CompanySponsored    15705
Other                 114
Name: AFFILIATION, dtype: int64

### USE_CASE

In [5]:
# Look at USE_CASE value counts for binning
application_df['USE_CASE'].value_counts()

Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: USE_CASE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of classifications to be replaced

use_cases_to_replace = list(application_df['USE_CASE'].value_counts().loc[lambda x: x < 5000].index)

# Replace in dataframe
for cls in use_cases_to_replace:
    application_df['USE_CASE'] = application_df['USE_CASE'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['USE_CASE'].value_counts()

Preservation    28095
ProductDev       5671
Other             533
Name: USE_CASE, dtype: int64

### ORGANIZATION


In [7]:
# Look at USE_CASE value counts for binning
application_df['ORGANIZATION'].value_counts()

Trust           23515
Association     10255
Co-operative      486
Corporation        43
Name: ORGANIZATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced

organization_to_replace = list(application_df['ORGANIZATION'].value_counts().loc[lambda x: x < 10000].index)

# Replace in dataframe
for cls in organization_to_replace:
    application_df['ORGANIZATION'] = application_df['ORGANIZATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['ORGANIZATION'].value_counts()

Trust          23515
Association    10255
Other            529
Name: ORGANIZATION, dtype: int64

### STATUS

In [9]:
# Look at USE_CASE value counts for binning
application_df['STATUS'].value_counts()

1    34294
0        5
Name: STATUS, dtype: int64

In [10]:
# Drop STATUS
application_df = application_df.drop(columns=['STATUS'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,N,5000,1
1,T3,Independent,C2000,Preservation,Other,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,N,6692,1
4,T3,Independent,C1000,Other,Trust,100000-499999,N,142590,1


### SPECIAL_CONSIDERATIONS

In [11]:
# Look at SPECIAL_CONSIDERATIONS value counts for binning
application_df['SPECIAL_CONSIDERATIONS'].value_counts()

N    34272
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64

In [12]:
# Drop STATUS
application_df = application_df.drop(columns=['SPECIAL_CONSIDERATIONS'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Other,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Other,Trust,100000-499999,142590,1


# Split and Scale

In [13]:
# Convert categorical data to numeric with `pd.get_dummies`
X = application_df.drop(columns='IS_SUCCESSFUL')
X = pd.get_dummies(X)

# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train,X_test,y_train,y_test = train_test_split(X,y)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# number of features
num_of_features = len(X_test_scaled[0])
num_of_features

34

## Keras Tuner

In [14]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Allow kerastuner to decide number of neurons in first hidden layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=3,
        max_value=43,
        step=5), activation=activation, input_dim = num_of_features))

    # Allow kerastuner to decide number of extra hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i+1),
            min_value=3,
            max_value=43,
            step=5),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation=activation))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [15]:
# Import the kerastuner library
!pip install keras-tuner --upgrade
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 3.2 MB/s eta 0:00:00


In [16]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 00m 55s]
val_accuracy: 0.4588921368122101

Best val_accuracy So Far: 0.7338775396347046
Total elapsed time: 00h 24m 08s


In [17]:
# Get top 3 model hyperparameters and print the values
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)

{'activation': 'sigmoid', 'first_units': 13, 'num_layers': 5, 'units_1': 33, 'units_2': 8, 'units_3': 8, 'units_4': 3, 'units_5': 23, 'tuner/epochs': 7, 'tuner/initial_epoch': 0, 'tuner/bracket': 1, 'tuner/round': 0}
{'activation': 'sigmoid', 'first_units': 13, 'num_layers': 5, 'units_1': 33, 'units_2': 8, 'units_3': 8, 'units_4': 3, 'units_5': 23, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 1, 'tuner/round': 1, 'tuner/trial_id': '0050'}
{'activation': 'sigmoid', 'first_units': 18, 'num_layers': 4, 'units_1': 8, 'units_2': 23, 'units_3': 23, 'units_4': 38, 'units_5': 23, 'tuner/epochs': 3, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0}


In [18]:
# Evaluate the top 3 models against the test dataset
top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5771 - accuracy: 0.7339 - 873ms/epoch - 3ms/step
Loss: 0.5771007537841797, Accuracy: 0.7338775396347046
268/268 - 1s - loss: 0.5770 - accuracy: 0.7329 - 1s/epoch - 4ms/step
Loss: 0.5770201683044434, Accuracy: 0.7329446077346802
268/268 - 1s - loss: 0.5772 - accuracy: 0.7324 - 1s/epoch - 4ms/step
Loss: 0.5771626830101013, Accuracy: 0.7323614954948425


# Compiling, Training and Evaluating the Model: TRIAL 1/3

In [19]:
top_hyper[0].values

{'activation': 'sigmoid',
 'first_units': 13,
 'num_layers': 5,
 'units_1': 33,
 'units_2': 8,
 'units_3': 8,
 'units_4': 3,
 'units_5': 23,
 'tuner/epochs': 7,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 1,
 'tuner/round': 0}

In [20]:
top_hyper[0].values['first_units']

13

In [21]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[0].values['first_units'], activation = top_hyper[0].values['activation'], input_dim = num_of_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[0].values['units_1'], activation = top_hyper[0].values['activation']))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[0].values['units_2'], activation = top_hyper[0].values['activation']))

# Forth hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[0].values['units_3'], activation = top_hyper[0].values['activation']))

# Fifth hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[0].values['units_4'], activation = top_hyper[0].values['activation']))

# Sixth hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[0].values['units_5'], activation = top_hyper[0].values['activation']))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation = top_hyper[0].values['activation']))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 13)                455       
                                                                 
 dense_7 (Dense)             (None, 33)                462       
                                                                 
 dense_8 (Dense)             (None, 8)                 272       
                                                                 
 dense_9 (Dense)             (None, 8)                 72        
                                                                 
 dense_10 (Dense)            (None, 3)                 27        
                                                                 
 dense_11 (Dense)            (None, 23)                92        
                                                                 
 dense_12 (Dense)            (None, 1)                

In [22]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics="accuracy")

In [23]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=20)

Epoch 1/20
804/804 [==============================] - 9s 8ms/step - loss: 0.6682 - accuracy: 0.5918
Epoch 2/20
804/804 [==============================] - 5s 6ms/step - loss: 0.5833 - accuracy: 0.7291
Epoch 3/20
804/804 [==============================] - 5s 6ms/step - loss: 0.5800 - accuracy: 0.7298
Epoch 4/20
804/804 [==============================] - 8s 9ms/step - loss: 0.5794 - accuracy: 0.7295
Epoch 5/20
804/804 [==============================] - 5s 6ms/step - loss: 0.5790 - accuracy: 0.7299
Epoch 6/20
804/804 [==============================] - 8s 9ms/step - loss: 0.5786 - accuracy: 0.7297
Epoch 7/20
804/804 [==============================] - 5s 7ms/step - loss: 0.5771 - accuracy: 0.7304
Epoch 8/20
804/804 [==============================] - 4s 4ms/step - loss: 0.5755 - accuracy: 0.7294
Epoch 9/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5743 - accuracy: 0.7292
Epoch 10/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5738 - accuracy: 0.7297

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5664 - accuracy: 0.7301 - 504ms/epoch - 2ms/step
Loss: 0.5664418935775757, Accuracy: 0.7301457524299622


In [25]:
# Export our model to HDF5 file
from google.colab import files
nn.save('/content/drive/My Drive/UC Berkeley Data Analytics Bootcamp/Module 21/Challenge/AlphabetSoupCharity_Optimization_1.h5')
files.download('/content/drive/My Drive/UC Berkeley Data Analytics Bootcamp/Module 21/Challenge/AlphabetSoupCharity_Optimization_1.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Compiling, Training and Evaluating the Model: TRIAL 2/3

In [26]:
top_hyper[1].values

{'activation': 'sigmoid',
 'first_units': 13,
 'num_layers': 5,
 'units_1': 33,
 'units_2': 8,
 'units_3': 8,
 'units_4': 3,
 'units_5': 23,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/bracket': 1,
 'tuner/round': 1,
 'tuner/trial_id': '0050'}

In [27]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[1].values['first_units'], activation = top_hyper[1].values['activation'], input_dim = num_of_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[1].values['units_1'], activation = top_hyper[1].values['activation']))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[1].values['units_2'], activation = top_hyper[1].values['activation']))

# Forth hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[1].values['units_3'], activation = top_hyper[1].values['activation']))

# Fifth hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[1].values['units_4'], activation = top_hyper[1].values['activation']))

# Sixth hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[1].values['units_5'], activation = top_hyper[1].values['activation']))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation = top_hyper[1].values['activation']))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 13)                455       
                                                                 
 dense_14 (Dense)            (None, 33)                462       
                                                                 
 dense_15 (Dense)            (None, 8)                 272       
                                                                 
 dense_16 (Dense)            (None, 8)                 72        
                                                                 
 dense_17 (Dense)            (None, 3)                 27        
                                                                 
 dense_18 (Dense)            (None, 23)                92        
                                                                 
 dense_19 (Dense)            (None, 1)                

In [28]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics="accuracy")

In [29]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=20)

Epoch 1/20
804/804 [==============================] - 3s 2ms/step - loss: 0.6726 - accuracy: 0.5817
Epoch 2/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5842 - accuracy: 0.7290
Epoch 3/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5792 - accuracy: 0.7307
Epoch 4/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5788 - accuracy: 0.7298
Epoch 5/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5781 - accuracy: 0.7310
Epoch 6/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5775 - accuracy: 0.7311
Epoch 7/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5767 - accuracy: 0.7298
Epoch 8/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5752 - accuracy: 0.7294
Epoch 9/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5737 - accuracy: 0.7312
Epoch 10/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5727 - accuracy: 0.7298

In [30]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5694 - accuracy: 0.7303 - 724ms/epoch - 3ms/step
Loss: 0.5694219470024109, Accuracy: 0.7302623987197876


In [31]:
# Export our model to HDF5 file
from google.colab import files
nn.save('/content/drive/My Drive/UC Berkeley Data Analytics Bootcamp/Module 21/Challenge/AlphabetSoupCharity_Optimization_2.h5')
files.download('/content/drive/My Drive/UC Berkeley Data Analytics Bootcamp/Module 21/Challenge/AlphabetSoupCharity_Optimization_2.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Compiling, Training and Evaluating the Model: TRIAL 3/3

In [32]:
top_hyper[2].values

{'activation': 'sigmoid',
 'first_units': 18,
 'num_layers': 4,
 'units_1': 8,
 'units_2': 23,
 'units_3': 23,
 'units_4': 38,
 'units_5': 23,
 'tuner/epochs': 3,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 2,
 'tuner/round': 0}

In [34]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[2].values['first_units'], activation = top_hyper[2].values['activation'], input_dim = num_of_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[2].values['units_1'], activation = top_hyper[2].values['activation']))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[2].values['units_2'], activation = top_hyper[2].values['activation']))

# Forth hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[2].values['units_3'], activation = top_hyper[2].values['activation']))

# Fifth hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[2].values['units_4'], activation = top_hyper[2].values['activation']))

# Sixth hidden layer
nn.add(tf.keras.layers.Dense(units=top_hyper[2].values['units_5'], activation = top_hyper[2].values['activation']))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation = top_hyper[2].values['activation']))

# Check the structure of the model
nn.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_25 (Dense)            (None, 18)                630       
                                                                 
 dense_26 (Dense)            (None, 8)                 152       
                                                                 
 dense_27 (Dense)            (None, 23)                207       
                                                                 
 dense_28 (Dense)            (None, 23)                552       
                                                                 
 dense_29 (Dense)            (None, 38)                912       
                                                                 
 dense_30 (Dense)            (None, 23)                897       
                                                                 
 dense_31 (Dense)            (None, 1)                

In [35]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics="accuracy")

In [36]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=20)

Epoch 1/20
804/804 [==============================] - 9s 4ms/step - loss: 0.6458 - accuracy: 0.6205
Epoch 2/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5787 - accuracy: 0.7280
Epoch 3/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5766 - accuracy: 0.7303
Epoch 4/20
804/804 [==============================] - 3s 3ms/step - loss: 0.5752 - accuracy: 0.7289
Epoch 5/20
804/804 [==============================] - 2s 3ms/step - loss: 0.5747 - accuracy: 0.7301
Epoch 6/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5741 - accuracy: 0.7290
Epoch 7/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5736 - accuracy: 0.7301
Epoch 8/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5724 - accuracy: 0.7301
Epoch 9/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5726 - accuracy: 0.7301
Epoch 10/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5717 - accuracy: 0.7298

In [37]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5633 - accuracy: 0.7273 - 488ms/epoch - 2ms/step
Loss: 0.5633372664451599, Accuracy: 0.7273469567298889


In [38]:
# Export our model to HDF5 file
from google.colab import files
nn.save('/content/drive/My Drive/UC Berkeley Data Analytics Bootcamp/Module 21/Challenge/AlphabetSoupCharity_Optimization_3.h5')
files.download('/content/drive/My Drive/UC Berkeley Data Analytics Bootcamp/Module 21/Challenge/AlphabetSoupCharity_Optimization_3.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>